# The network energy intensity of video streaming over Wi-Fi and 4G

**Authors:** David Mytton, Iain Staffell, Malte Jansen.

**Institution:** Centre for Environmental Policy, Imperial College London, London, SW7 1NE, UK.

**Correspondence:** <david@davidmytton.co.uk>.

## Summary

> Between 2010-2018 internet traffic grew ten-fold and is expected to double again by 2022. With video streaming making up 60% of that traffic, 65% on mobile, it is important to understand its energy consumption. Here we provide updated figures for the energy consumption of different sections of the internet by using observations from volunteers of home internet routing over Wi-Fi and 4G connections, then use survey and industry estimates to calculate the network energy intensity of video streaming. We estimate the 2019 network use-stage energy footprint of the UK’s 39 billion hours of video streaming at 4.2 TWh, or 1.3% of total electricity generation. We show that the network energy intensity of video streaming over Wi-Fi is 0.091 kWh/hour, compared to 0.207 kWh/hour over 4G mobile internet.

## This notebook

This notebook walks through the methodology and calculations as described in the accompanying paper.

#### Imports

In [1]:
%pip install -r requirements.txt

import numpy as np
import pandas as pd
import pint
import plotly.express as px
from pint import UnitRegistry

ureg = UnitRegistry()

Note: you may need to restart the kernel to use updated packages.


## Internet video traffic

We use YouTube as a case study because it is the largest video streaming service by traffic so has the most data available. We use various sources to calculate the total amount of time spent and data traffic volume. Assuming YouTube is representative of all video streaming, we then scale those numbers to video streaming in general.

We therefore start with an assessment of YouTube.

### YouTube watch time

YouTube does not publish statistics so we use figures from the advertising industry<sup>1</sup>, Ofcom<sup>2</sup>, and Netflix<sup>3</sup>. Using the total annual watch minutes, the total time watched on mobile devices and the total time watched on mobile networks, we can calculate the total time spent on Wi-Fi and non-mobile devices.

In [2]:
# Calculate the total amount of time spent watching YouTube on all
# networks and all devices. This is based on the total watch time during Dec 2019
# from advertising industry figures, and Ofcom figures for the time spent
# watching on a mobile device
def calculate_youtube_time_allnetworks_alldevices():
    # Dec 2019 UK YouTube minutes (mobile devices)
    youtube_time_mobile_dec = 32_099_000_000  # Source [1] (Pinpoint: pg 30)

    # What % watch YouTube on a mobile devices
    # Note this includes the use of mobile devices on both mobile and Wi-Fi networks
    youtube_percentage_mobiledevices = 0.73  # 73% Source [2] (Pinpoint: Figure 4.10, pg116)

    # Assume Dec 2019 figure is representative of the full year
    return (youtube_time_mobile_dec * 12) / youtube_percentage_mobiledevices

# Calculate the total amount of time spent watching YouTube on
# mobile networks.
def calculate_youtube_time_mobilenetworks(youtube_time_allnetworks_alldevices):
    # "Globally throughout the day, Netflix users' streaming on mobile networks 
    # hovers near or below 25 percent of total streaming"
    youtube_percentage_mobilenetworks = np.random.triangular(0.20, 0.25, 0.26)  # Source [3]

    return youtube_percentage_mobilenetworks * youtube_time_allnetworks_alldevices

# Calculate the total amount of time spent watching YouTube on Wi-Fi
# networks on all devices
def calculate_youtube_time_wifinetworks_alldevices(
        youtube_time_allnetworks_alldevices,
        youtube_time_mobilenetworks):
    return youtube_time_allnetworks_alldevices - youtube_time_mobilenetworks

# Run Monte Carlo simulation
N = 10_000  # Number of executions
youtube_time_allnetworks_alldevices = np.zeros(N)
youtube_time_mobilenetworks = np.zeros(N)
youtube_time_wifinetworks_alldevices = np.zeros(N)
for i in range(N):
    youtube_time_allnetworks_alldevices[i] = calculate_youtube_time_allnetworks_alldevices()
    youtube_time_mobilenetworks[i] = calculate_youtube_time_mobilenetworks(youtube_time_allnetworks_alldevices[i])
    youtube_time_wifinetworks_alldevices[i] = calculate_youtube_time_wifinetworks_alldevices(
        youtube_time_allnetworks_alldevices[i], 
        youtube_time_mobilenetworks[i])

# Display histograms
fig = px.box(youtube_time_mobilenetworks, y=0)
fig.update_layout(
    title_text='Annual UK 2019 YouTube minutes (mobile networks)',
    yaxis_title_text='Minutes'
)
fig.show()

fig = px.box(youtube_time_wifinetworks_alldevices, y=0)
fig.update_layout(
    title_text='Annual UK 2019 YouTube minutes (Wi-Fi networks, all devices)',
    yaxis_title_text='Minutes'
)
fig.show()

# Display mean averages
print(f'Annual UK 2019 YouTube minutes (all networks, all devices): {youtube_time_allnetworks_alldevices.mean():,.0f}')
print(f'Annual UK 2019 YouTube minutes (mobile networks): {youtube_time_mobilenetworks.mean():,.0f}')
print(f'Annual UK 2019 YouTube minutes (Wi-Fi networks, all devices): {youtube_time_wifinetworks_alldevices.mean():,.0f}')

Annual UK 2019 YouTube minutes (all networks, all devices): 527,654,794,521
Annual UK 2019 YouTube minutes (mobile networks): 124,843,500,542
Annual UK 2019 YouTube minutes (Wi-Fi networks, all devices): 402,811,293,979


### YouTube data volume

The amount of data transmitted during a single YouTube video streaming session varies based on factors such as device type, screen size, video resolution, framerate, bit rate, network speed and which formats the video was encoded into. 

For each video quality setting, we take the mean average of the range of values reported in an observational assessment of 1 hour of streaming<sup>4</sup>. In this assessment, data volume was recorded using three separate tools: Android's built-in data monitoring, Google's Datally app and the GlassWire data monitoring app.

In [3]:
def get_data_volume(video_quality=720):
    N = 10_000
    sim = np.zeros(N)
    for i in range(N):
        if video_quality == 480:
            data_volume = np.random.uniform(0.48, 0.66)  # 480p SD [min, max]
        elif video_quality == 720:
            data_volume = np.random.uniform(1.2, 2.7)  # 720p HD [min, max]
        elif video_quality == 1080:
            data_volume = np.random.uniform(2.5, 4.1)  # 1080p FHD [min ,max]
        elif video_quality == 1440:
            data_volume = np.random.uniform(2.7, 8.1)  # 1440p QHD [min ,max]
        elif video_quality == 2160:
            data_volume = np.random.uniform(5.5, 23.0)  # 2160p UHD 4k [min ,max]
    
        sim[i] = data_volume
    
    return sim

### Video traffic

We need to know how much traffic is from YouTube and video in general for both mobile and non-mobile.

In [4]:
internetTrafficYouTube = 0.118 # 11.8% Source [5] (Pinpoint: pg12)
internetTrafficVideo = 0.60 # 60% Source [5] (Pinpoint: pg6)
mobileTrafficYouTube = 0.271 # 27.1% Source [6] (Pinpoint: pg9)
mobileTrafficVideo = 0.655 # 65.5% Source [6] (Pinpoint: pg5)

## Video streaming time and data

Using the figures above, we calculate the total annual time spent and associated data volume for YouTube.

We assume mobile traffic is all 4G and non-mobile traffic is all Wi-Fi because 97% of UK premises have access to decent fixed (at least 10 Mbit/s) and 4G (at least 2 Mbit/s) services, 91% of geographic areas of the UK are able to receive 4G data service from at least one operator, and 4G carries 90% of UK mobile data traffic<sup>7</sup>. 3G connectivity is excluded because it carries only a small proportion of data traffic.

We assume the default video quality is 720p High Definition (HD) because during the 2020 Coronavirus pandemic, the default was reduced to 480p Standard Definition (SD)<sup>8</sup>. The reference year for this assessment is 2019.

In [5]:
# 720p HD video quality
video_quality = 720
hour_data_volume = get_data_volume(video_quality)

fig = px.box(hour_data_volume, y=0)
fig.update_layout(
    title_text=f'Data volume generated by 1 hour YouTube streaming at {video_quality}p',
    yaxis_title_text='Gigabytes (GB)'
)
fig.show()

### YouTube

In [6]:
# Tidy up the calculations into an easily readable data frame
youtube = pd.DataFrame({
    'Time': {
        '4G': youtube_time_mobilenetworks.mean() * ureg.minutes,
        'Wi-Fi': youtube_time_wifinetworks_alldevices.mean() * ureg.minutes,
        'Total': (youtube_time_mobilenetworks.mean() + youtube_time_wifinetworks_alldevices.mean()) * ureg.minutes,
    },
    'Data': {
        '4G': (youtube_time_mobilenetworks.mean() / 60 * hour_data_volume.mean()) * ureg.gigabytes,
        'Wi-Fi': (youtube_time_wifinetworks_alldevices.mean() / 60 * hour_data_volume.mean()) * ureg.gigabytes,
        'Total': ((youtube_time_mobilenetworks.mean() * hour_data_volume.mean()) / 60 + (youtube_time_wifinetworks_alldevices.mean() / 60 * hour_data_volume.mean())) * ureg.gigabytes,
    }
})

# Format nicely for display
youtube_display = pd.DataFrame({
    'Time': {
        '4G': f'{youtube["Time"]["4G"].to("hour").to_compact().magnitude:,.1f} bn hours',
        'Wi-Fi': f'{youtube["Time"]["Wi-Fi"].to("hour").to_compact().magnitude:,.1f}  bn hours',
        'Total': f'{youtube["Time"]["Total"].to("hour").to_compact().magnitude:,.1f} bn hours',

    },
    'Data': {
        '4G': f'{youtube["Data"]["4G"].to("exabyte"):,.1f~H}',
        'Wi-Fi': f'{youtube["Data"]["Wi-Fi"].to("exabyte"):,.1f~H}',
        'Total': f'{youtube["Data"]["Total"].to("exabyte"):,.1f~H}',

    }
})
youtube_display

,Time,Data
4G,2.1 bn hours,4.0 EB
Wi-Fi,6.7 bn hours,13.1 EB
Total,8.8 bn hours,17.1 EB


In [7]:
youtube_time = pd.DataFrame({
    '4G': youtube_time_mobilenetworks,
    'Wi-Fi': youtube_time_wifinetworks_alldevices,
})

fig = px.box(youtube_time)
fig.update_layout(
    title_text=f'Annual UK 2019 YouTube minutes',
    yaxis_title_text='Minutes',
    xaxis_title_text='Connection type'
)
fig.show()

In [8]:
youtube_data = pd.DataFrame({
    '4G': ((youtube_time_mobilenetworks / 60) * hour_data_volume),
    'Wi-Fi': (youtube_time_wifinetworks_alldevices / 60) * hour_data_volume,
})

fig = px.box(youtube_data)
fig.update_layout(
    title_text=f'Annual UK 2019 YouTube data volume',
    yaxis_title_text='Gigabytes (GB)',
    xaxis_title_text='Connection type'
)
fig.show()

#### Validation by comparison

An Ofcom survey<sup>2</sup> reported the number of unique visitors to YouTube out of the UK population and how much time they spent on it per day in 2019. From this we can calculate the total viewing time and compare to our figure.

In [9]:
# YouTube daily visitors
ofcom_youtube_daily_visitors = 41_970_000 # Source [2] (Pinpoint: Figure 4.6, pg111)

# YouTube average daily time spent
ofcom_youtube_average_daily_time = 35 * ureg.minutes # Source [2] (Pinpoint: Figure 4.8, pg 114)

# Annual UK YouTube time spent
ofcom_youtube_annual_time = ((ofcom_youtube_average_daily_time * 365) * ofcom_youtube_daily_visitors)
print(f'Ofcom survey: {ofcom_youtube_annual_time.to("hours").magnitude:,.0f} ({ofcom_youtube_annual_time.to("hours").to_compact().magnitude:,.1f}bn) hours')
print(f'Our calculation: {youtube["Time"]["Total"].to("hours").magnitude:,.0f} ({youtube["Time"]["Total"].to("hours").to_compact().magnitude:,.1f}bn) hours')

# % difference
difference = ((youtube['Time']['Total'] - ofcom_youtube_annual_time) / youtube['Time']['Total']) * 100
print(f'Difference: {difference.magnitude:,.1f}%')

Ofcom survey: 8,936,112,500 (8.9bn) hours
Our calculation: 8,794,246,575 (8.8bn) hours
Difference: -1.6%


### All UK video streaming

Given that YouTube is a certain percentage of all internet traffic, and we know what percentage of traffic is video streaming, if we assume that YouTube is representative of all video streaming we can extrapolate for all UK video streaming.

In [10]:
# Extrapolate to all video streaming
allukvideo_data_4g = (youtube_data['4G'] / mobileTrafficYouTube) * mobileTrafficVideo
allukvideo_data_wifi = (youtube_data['Wi-Fi'] / internetTrafficYouTube) * internetTrafficVideo
allukvideo_time_4g = allukvideo_data_4g / hour_data_volume.mean()
allukvideo_time_wifi = allukvideo_data_wifi / hour_data_volume.mean()

# Create a dataframe for easy analysis
allukvideo_time = pd.DataFrame({
    '4G': allukvideo_time_4g,
    'Wi-Fi': allukvideo_time_wifi,
})

# This is separate so the dataframe can be passed to Plotly
allukvideo_time_total = (allukvideo_time_4g.mean() * ureg.hours) + \
    (allukvideo_time_wifi.mean() * ureg.hours)

allukvideo_data = pd.DataFrame({
    '4G': allukvideo_data_4g,
    'Wi-Fi': allukvideo_data_wifi,
})

# This is separate so the dataframe can be passed to Plotly
allukvideo_data_total = (allukvideo_data_4g.mean() * ureg.gigabytes) + \
    (allukvideo_data_wifi.mean() * ureg.gigabytes)

In [11]:
fig = px.box(allukvideo_time)
fig.update_layout(
    title_text=f'Annual UK 2019 video streaming minutes',
    yaxis_title_text='Hours',
    xaxis_title_text='Connection type'
)
fig.show()

In [12]:
fig = px.box(allukvideo_data)
fig.update_layout(
    title_text=f'Annual UK 2019 video streaming data volume',
    yaxis_title_text='Gigabytes (GB)',
    xaxis_title_text='Connection type'
)
fig.show()

#### Validation by comparison

An Ofcom survey<sup>9</sup> reported the total number of video minutes watched per day per person and the proportion of that which was not broadcast content i.e. online video streaming vs broadcast TV. Based on the UK population<sup>10</sup>, we can calculate the total viewing time and compare to our figure.

In [13]:
ofcom_timeonline_perperson = 209 * ureg.minutes # Source [2] (Pinpoint: Paragraph, pg 9)

ofcom_videotime_daily_perperson = 294  * ureg.minutes # Source [9] (Pinpoint: Figure 1.4, pg 16)
ofcom_percentage_nonbroadcast = 0.31 # Source [9] (Pinpoint: Figure 1.4, pg 16)
uk_population = 66_400_000 # Source [10]

ofcom_time_streaming = (ofcom_videotime_daily_perperson * ofcom_percentage_nonbroadcast * uk_population * 365)
our_time_streaming = (allukvideo_time['4G'].mean() + allukvideo_time['Wi-Fi'].mean()) * ureg.hours

print(f'Ofcom survey: {ofcom_time_streaming.to("hours").magnitude:,.0f} ({ofcom_time_streaming.to("hours").to_compact().magnitude:,.1f}bn) hours')
print(f'Our calculation: {our_time_streaming.to("hours").magnitude:,.0f} ({our_time_streaming.to("hours").to_compact().magnitude:,.1f}bn) hours')

# % difference
difference = ((our_time_streaming -
              ofcom_time_streaming) / our_time_streaming) * 100
print(f'Difference: {difference.magnitude:,.1f}%')

Ofcom survey: 36,814,484,000 (36.8bn) hours
Our calculation: 39,164,797,925 (39.2bn) hours
Difference: 6.0%


## Traceroute sample analysis

116 Scamper traceroute samples were returned by 29 participants (see main paper for background and recruitment methods). The individual samples are provided in `/traceroute-samples/samples/`. An aggregated CSV is provided in `/traceroute-samples/traceroute-samples.csv` with network ownership metadata returned by [IPInfo](https://ipinfo.io). Two samples returned anomalous results where zero hops were reported - these are excluded as `NaN`. 

See `/traceroute-samples/aggregate-samples.ipynb` for the aggregation code.

In [14]:
traceroutes = pd.read_csv('../traceroute-samples/traceroute-samples.csv')

# ASN Names
asn_google = 'AS15169 Google LLC'
asn_fb = 'AS32934 Facebook, Inc.'

# Destination ASN = 4G
traceroutes_4g = traceroutes.query('Connection == "4g"')
traceroutes_wifi = traceroutes.query('Connection == "wifi"')

# Destination ASN = ISP
traceroutes_isp_all = traceroutes.query(
    '`Destination ASN` != @asn_google and `Destination ASN` != @asn_fb')
traceroutes_isp_4g = traceroutes_isp_all.query('Connection == "4g"')
traceroutes_isp_wifi = traceroutes_isp_all.query('Connection == "wifi"')

# Destination ASN = Google or Facebook
traceroutes_googfb_all = traceroutes.query(
    '`Destination ASN` == @asn_google or `Destination ASN` == @asn_fb')
traceroutes_googfb_4g = traceroutes_googfb_all.query('Connection == "4g"')
traceroutes_googfb_wifi = traceroutes_googfb_all.query('Connection == "wifi"')

### Hop counts

Calculate the mean hop count and standard deviation based on the owner of the destination network. The ISP owning the destination network indicates a caching device is deployed within the network.

In [15]:
hopCounts = pd.DataFrame({
    'Count': {  # Count total number of samples
        'All': traceroutes['Participant City'].count(),
        'Connection: 4G': traceroutes_4g['Participant City'].count(),
        'Connection: Wi-Fi': traceroutes_wifi['Participant City'].count(),
        'Destination - ISP: All': traceroutes_isp_all['Participant City'].count(),
        'Destination - ISP: 4G': traceroutes_isp_4g['Participant City'].count(),
        'Destination - ISP: Wi-Fi': traceroutes_isp_wifi['Participant City'].count(),
        'Destination - GOOG or FB: All': traceroutes_googfb_all['Participant City'].count(),
        'Destination - GOOG or FB: 4G': traceroutes_googfb_4g['Participant City'].count(),
        'Destination - GOOG or FB: Wi-Fi': traceroutes_googfb_wifi['Participant City'].count(),
    },
    'Mean': {  #  Mean hop count, excluding anomalous results
        'All': traceroutes['Trace Hop Count'].mean(skipna=True),
        'Connection: 4G': traceroutes_4g['Trace Hop Count'].mean(skipna=True),
        'Connection: Wi-Fi': traceroutes_wifi['Trace Hop Count'].mean(skipna=True),
        'Destination - ISP: All': traceroutes_isp_all['Trace Hop Count'].mean(skipna=True),
        'Destination - ISP: 4G': traceroutes_isp_4g['Trace Hop Count'].mean(skipna=True),
        'Destination - ISP: Wi-Fi': traceroutes_isp_wifi['Trace Hop Count'].mean(skipna=True),
        'Destination - GOOG or FB: All': traceroutes_googfb_all['Trace Hop Count'].mean(skipna=True),
        'Destination - GOOG or FB: 4G': traceroutes_googfb_4g['Trace Hop Count'].mean(skipna=True),
        'Destination - GOOG or FB: Wi-Fi': traceroutes_googfb_wifi['Trace Hop Count'].mean(skipna=True),
    },
    'StdDev': {  #  Hop count standard deviation, excluding anomalous results
        'All': traceroutes['Trace Hop Count'].std(skipna=True),
        'Connection: 4G': traceroutes_4g['Trace Hop Count'].std(skipna=True),
        'Connection: Wi-Fi': traceroutes_wifi['Trace Hop Count'].std(skipna=True),
        'Destination - ISP: All': traceroutes_isp_all['Trace Hop Count'].std(skipna=True),
        'Destination - ISP: 4G': traceroutes_isp_4g['Trace Hop Count'].std(skipna=True),
        'Destination - ISP: Wi-Fi': traceroutes_isp_wifi['Trace Hop Count'].std(skipna=True),
        'Destination - GOOG or FB: All': traceroutes_googfb_all['Trace Hop Count'].std(skipna=True),
        'Destination - GOOG or FB: 4G': traceroutes_googfb_4g['Trace Hop Count'].std(skipna=True),
        'Destination - GOOG or FB: Wi-Fi': traceroutes_googfb_wifi['Trace Hop Count'].std(skipna=True),
    }
})

hopCounts

,Count,Mean,StdDev
All,116,7.412281,4.244333
Connection: 4G,58,7.362069,5.000696
Connection: Wi-Fi,58,7.464286,3.330107
Destination - ISP: All,54,6.096154,2.443556
Destination - ISP: 4G,20,7.600000,2.722228
Destination - ISP: Wi-Fi,34,5.156250,1.705955
Destination - GOOG or FB: All,62,8.516129,5.065932
Destination - GOOG or FB: 4G,38,7.236842,5.888304
Destination - GOOG or FB: Wi-Fi,24,10.541667,2.302724


Mean and standard deviation for hop count where the destination network belongs to the ISP:

In [16]:
print(f'{hopCounts["Mean"]["Destination - ISP: All"]:.1f} ± {hopCounts["StdDev"]["Destination - ISP: All"]:.1f} (from n = {hopCounts["Count"]["Destination - ISP: All"]:.0f} samples)')

6.1 ± 2.4 (from n = 54 samples)


Mean and standard deviation for hop count where the destination network belongs to Facebook or Google:

In [17]:
print(f'{hopCounts["Mean"]["Destination - GOOG or FB: All"]:.1f} ± {hopCounts["StdDev"]["Destination - GOOG or FB: All"]:.1f} (from n = {hopCounts["Count"]["Destination - GOOG or FB: All"]:.0f} samples)')

8.5 ± 5.1 (from n = 62 samples)


### Apportion by connection type

We want to apportion the traffic so it is representative of the routing revealed by the traceroutes. This means we need to calculate what percentage of traffic terminates at the ISP for 4G and Wi-Fi.

In [18]:
traffic_isp_4g = hopCounts['Count']['Destination - ISP: 4G'] / hopCounts['Count']['Connection: 4G']
traffic_googfb_4g = hopCounts['Count']['Destination - GOOG or FB: 4G'] / hopCounts['Count']['Connection: 4G']
print(f'4G traffic: {traffic_isp_4g:.0%} terminates at the ISP and {traffic_googfb_4g:.0%} terminates at Google or Facebook.')

traffic_isp_wifi = hopCounts['Count']['Destination - ISP: Wi-Fi'] / hopCounts['Count']['Connection: Wi-Fi']
traffic_googfb_wifi = hopCounts['Count']['Destination - GOOG or FB: Wi-Fi'] / hopCounts['Count']['Connection: Wi-Fi']
print(f'Wi-Fi traffic: {traffic_isp_wifi:.0%} terminates at the ISP and {traffic_googfb_wifi:.0%} terminates at Google or Facebook.')

4G traffic: 34% terminates at the ISP and 66% terminates at Google or Facebook.
Wi-Fi traffic: 59% terminates at the ISP and 41% terminates at Google or Facebook.


## Internet energy

The internet consists of multiple sections which connect the end-user device to the data center, so we must assess them each separately.

The end-user device is included for 4G connections because radio transmission is built into the device used for watching and therefore cannot be excluded. Over Wi-Fi, the many different types of end-user device mean the energy consumption is highly variable and would therefore confuse any estimates of use-stage network energy because the device is not a logical part of the network. The end-user device is therefore excluded for Wi-Fi connections.

The internet is split into three sections: Edge, Metro, and Core, which are made up of multiple components such as switches, routers, and fiber multiplexers, each with different deployment characteristics such as number of switches/routers, and utilization values.

The data center, caching nodes and end-user device on Wi-Fi networks are excluded, see System boundaries in the main paper for reasoning.

Here we calculate the use-stage network transmission energy intensity of each section.

#### Units

Define new `pint` units:

In [19]:
ureg.define('watthour_hour = watthour / hour = Wh/hour')
ureg.define('kilowatthour_hour = kilowatthour / hour = kWh/hour')
ureg.define('kilowatthour_gb = kilowatthour / gigabyte = kWh/GB')
ureg.define('joules_gigabit = joules / gigabit = J/Gb')
ureg.define('kilowatthours_gigabyte = kilowatt hours / gigabyte = kWh/GB')

### CPE & AN - Wi-Fi

For Wi-Fi, we follow the approach in Coroama et al.<sup>11</sup> and Schien et al.<sup>12</sup> with calculations from the "Internet video traffic" section above and updated PUE figures from Masanet et al.<sup>13</sup>.

#### CPE

In [20]:
# Total time equipment is on 
# (Pinpoint: pg16 in [11], citing Table 7-1 in [14])
cpe_wifi_t_on = 1440 * ureg.minutes

# Total time in which the router is in use
# Calculated based on the total amount of time spent online from Ofcom survey [2]
# Apportioned based on the total amount of time spent video streaming on Wi-Fi
# vs mobile.
cpe_wifi_t_use = ofcom_timeonline_perperson - \
    (ofcom_timeonline_perperson * mobileTrafficYouTube)

# Total idle time when the router is on but not used
cpe_wifi_t_idle = cpe_wifi_t_on - cpe_wifi_t_use

# Power of the router
# (Pinpoint: pg15 in [11])
cpe_wifi_router_power = 8 * ureg.watts

#### AN

In [21]:
# Redundancy
# (Pinpoint: pg12 in [11])
an_wifi_r = 2

# Power of the access network devices
# (Pinpoint: pg15 in [11], citing pg5 (804) in [12].)
an_wifi_power = 2 * ureg.watts

# Number of users connected to access devices
# (Pinpoint: pg14 in [11])
an_wifi_users = 1

# PUE of the telecoms site
# (Pinpoint: [13] Supplementary Material, Sheet "Regional PUE" - Traditional 
# Data center PUE Western Europe 2019.)
an_wifi_pue = 1.99

#### CPE & AN

In [22]:
i_cpe_an_wifi = (1 + cpe_wifi_t_idle / cpe_wifi_t_use) * cpe_wifi_router_power + \
    ((an_wifi_power / an_wifi_users) * an_wifi_r) * an_wifi_pue
i_cpe_an_wifi = (i_cpe_an_wifi.magnitude) * ureg.watthour_hour
i_cpe_an_wifi.to('kilowatthour_hour')

0.08356990017130368 <Unit('kilowatthour_hour')>

### CPE & AN - 4G

For 4G AN, Pihkola et al.<sup>15</sup> is the most up to date peer-reviewed figure – 0.1 kWh/GB. 

A figure for 4G CPE is more difficult to calculate because the equivalent of the Wi-Fi router is embedded in the device used for watching, so the end-user device must be included. There are many different types of phone or tablet with varying battery capacities. A simplified approach was taken by using data from Apple for the iPhone 11 Pro<sup>16,17</sup>. Apple reports that a battery charged to 100% will support 11 hours of streamed video<sup>18</sup>. We assume assuming battery consumption is linear and that 100% of the battery is depleted after 11 hours.

#### CPE

In [23]:
iphone_capacity = 11.67 * ureg.watthours
streaming_hours = 11 * ureg.hours

cpe_4g = iphone_capacity / streaming_hours
cpe_4g

1.0609090909090908 <Unit('watt_hour / hour')>

#### AN

In [24]:
an_4g = 0.1 * ureg.kilowatthour_gb # Source [15]
an_4g

0.1 <Unit('kilowatthour_gb')>

#### CPE & AN

Unlike with Wi-Fi, we can't combine 4G CPE & AN here because the units are different, so this is done later.

### Edge, Metro, Core

For the sections of the internet - Edge, Metro and Core - we follow the approach in Schien et al.<sup>12</sup> and [associated implementation details](https://nbviewer.jupyter.org/gist/dschien/24bbb049ba9be347fc22), with some modifications.

The assumption of fiber Access Network connectivity used by Schien et al.<sup>12</sup> in 2015 was criticized by Aslan et al.<sup>19</sup>. However, we assume this is now appropriate for the UK in 2019 where 95% of premises have access to superfast (>30Mbit/s) and 53% have access to ultrafast (300Mbit/s) internet connectivity<sup>7</sup>, both of which use fiber-based technologies such as Fiber to the Cabinet (FTTC).

#### Number of routers

Schien et al.<sup>12</sup> assumes 6 routers in total with 4 routers described as being in the "long haul" network, which we assume to mean "Core" network. Instead, we use our traceroute results (above) and allocate across Metro and Core in the same proportion, assuming Edge always has 1 router. We follow the approach of Schien et al.<sup>12</sup> where the number of routers in a route of `n` hops is `n + 1`.

In [25]:
# Original number of routers from [12]
# (Pinpoint: pg7 in [12])
routers_original = {}
routers_original['edge'] = 1
routers_original['metro'] = 1
routers_original['core'] = 4
routers_original['total'] = routers_original['edge'] + routers_original['metro'] + routers_original['core']

# Adjusted number based on traceroute results
routers_adjusted = {}
routers_adjusted['edge'] = 1 # Always 1
routers_adjusted['metro'] = routers_original['metro'] / routers_original['total'] * (traceroutes['Trace Hop Count'].mean(skipna=True) + 1)
routers_adjusted['core'] = routers_original['core'] / routers_original['total'] * (traceroutes['Trace Hop Count'].mean(skipna=True) + 1)
routers_adjusted['total'] = routers_adjusted['edge'] + routers_adjusted['metro'] + routers_adjusted['core']
routers_adjusted

{'edge': 1,
 'metro': 1.402046783625731,
 'core': 5.608187134502924,
 'total': 8.010233918128655}

#### Edge

In [26]:
# Redundancy
# (Pinpoint: pg7 in [12])
edge_r = 2

# PUE of the telecoms site
# (Pinpoint: [13] Supplementary Material, Sheet "Regional PUE" - Traditional 
# Data center PUE Western Europe 2019.)
edge_pue = 1.99

# Edge overcapacity factor (% utilisation)
# This is not provided Schien et al. 27 so 
# we assume the value is the same as the Metro network
# (Pinpoint: pg7 in [12])
edge_overcapacity = 6.67

# Edge switch energy intensity
# (Pinpoint: pg6 in [12])
edge_switch_i = 8 * ureg.joules_gigabit

# Edge router energy intensity
# (Pinpoint: pg6 in [12])
edge_router_i = 64 * ureg.joules_gigabit

# Energy intensity of edge
i_edge = edge_r * edge_pue * edge_overcapacity * (edge_switch_i + edge_router_i)
i_edge

1911.3552 <Unit('joules_gigabit')>

#### Metro

In [27]:
# Redundancy
# (Pinpoint: pg7 in [12])
metro_r = 2

# PUE of the telecoms site
# (Pinpoint: [13] Supplementary Material, Sheet "Regional PUE" - Traditional
# Data center PUE Western Europe 2019.)
metro_pue = 1.99

# Metro overcapacity factor (% utilisation)
# (Pinpoint: pg7 in [12])
metro_overcapacity = 6.67

# Number of metro routers
metro_routers = routers_adjusted['metro']

# Energy intensity per metro router
# (Pinpoint: pg7 in [12])
metro_router_i = 39 * ureg.joules_gigabit

# Energy intensity of metro transmission
# (Pinpoint: pg7 in [12] - "We assume a cumulative, nominal energy intensity of
# the optical transmission system to vary between 230, 147 and 316 J/Gb
# (average, 25th, 75th percentile) for metro"
metro_transmission_i = 230 * ureg.joules_gigabit

# Energy intensity of metro
i_metro = metro_pue * metro_overcapacity * \
    (metro_r * metro_routers * metro_router_i + metro_transmission_i)
i_metro

4504.422430701754 <Unit('joules_gigabit')>

#### Core

In [28]:
# Redundancy
# (Pinpoint: pg7 in [12])
core_r = 2

# PUE of the telecoms site
# (Pinpoint: [13] Supplementary Material, Sheet "Regional PUE" - Traditional 
# Data center PUE Western Europe 2019.)
core_pue = 1.99

# Metro overcapacity factor (% utilisation)
# (Pinpoint: pg7 in [12])
core_overcapacity = 3.03

# Number of core routers
core_routers = routers_adjusted['core']

# Energy intensity per core router
# (Pinpoint: pg7 in [12])
core_router_i = 26.7 * ureg.joules_gigabit

# Energy intensity of core transmission
# (Pinpoint: pg7 in [12] - ""We assume a cumulative, nominal energy intensity 
# of the optical transmission system to vary between  1593, 893 and 2292 J/Gb 
# (average, 25th, 75th percentile) for core networks""
core_transmission_i = 1593 * ureg.joules_gigabit

# Energy intensity of core
i_core = core_pue * core_overcapacity * (core_r * core_routers * core_router_i + core_transmission_i)
i_core

11411.069730526313 <Unit('joules_gigabit')>

### Internet

The equipment energy intensity figures used above are taken from Schien et al.<sup>12</sup>, however these are from 2014. The industry suffers from a lack of published data about equipment energy intensity, and it was not possible to find more recent numbers. We therefore apply an efficiency improvement adjustment to the energy intensity for each network component. Future work could survey ISPs to inventory deployed equipment, take energy measurements and produce more accurate data. In lieu of such data, we apply a reduction of 79.82% to the energy intensity based on the expected decrease demonstrated by Aslan et al.<sup>19</sup>.

In [29]:
# Decrease expected from 2014-2019
expected_decrease = -0.7982

# Calculate decrease
i_edge_2019 = i_edge * (1 + expected_decrease)
i_metro_2019 = i_metro * (1 + expected_decrease)
i_core2019 = i_core * (1 + expected_decrease)

i_internet = {
    'edge': i_edge_2019.to('kilowatthours_gigabyte'),
    'metro': i_metro_2019.to('kilowatthours_gigabyte'),
    'core': i_core2019.to('kilowatthours_gigabyte'),
}
i_internet['total'] = i_internet['edge'] + i_internet['metro'] + i_internet['core']

print(f'Edge: {i_internet["edge"]:.4f~H}')
print(f'Metro: {i_internet["metro"]:.4f~H}')
print(f'Core: {i_internet["core"]:.4f~H}')
print(f'Total: {i_internet["total"]:.4f~H}')

Edge: 0.0009 kWh/GB
Metro: 0.0020 kWh/GB
Core: 0.0051 kWh/GB
Total: 0.0080 kWh/GB


## Video streaming energy

Using the above values, we can now calculate the total use-stage network energy consumption for all video streaming in the UK for 2019.

### CPE & AN

All traffic flows through the CPE & AN, so we calculate the total energy consumption for this section.

In [30]:
cpe_an_4g = (an_4g * (allukvideo_data['4G'].mean() * ureg.gigabytes)) + \
    (cpe_4g * (allukvideo_time['4G'].mean() * ureg.hours))
print(f'CPE & AN - 4G: {cpe_an_4g.to("gigawatt hours").magnitude:,.0f} GWh')

cpe_an_wifi = i_cpe_an_wifi * (allukvideo_time['Wi-Fi'].mean() * ureg.hours)
print(f'CPE & AN - Wi-Fi: {cpe_an_wifi.to("gigawatt hours").magnitude:,.0f} GWh')

cpe_an_total = cpe_an_4g + cpe_an_wifi
print(f'CPE & AN: {cpe_an_total.to("gigawatt hours").magnitude:,.0f} GWh')

CPE & AN - 4G: 983 GWh
CPE & AN - Wi-Fi: 2,853 GWh
CPE & AN: 3,836 GWh


### Caching nodes

The traceroute samples reveal that network traffic can have two destination networks:

1. A destination network owned by the ISP. We assume this means the ISP has a caching device deployed within their network and so traffic traverses the CPE & AN and Edge network sections.
2. A destination network owned by the content provider (Google or Facebook in our tests). We assume this means traffic traverses all sections of the network - CPE & AN, Edge, Metro and Core.

We therefore apportion traffic based on the percentages above.

In [31]:
traffic_destination = pd.DataFrame({
    '4G': {
        'Traffic to ISP edge cache': traffic_isp_4g * (allukvideo_data['4G'].mean() * ureg.gigabyte),
        'Traffic to content provider': traffic_googfb_4g * (allukvideo_data['4G'].mean() * ureg.gigabyte),
    },
    'Wi-Fi': {
        'Traffic to ISP edge cache': traffic_isp_wifi * (allukvideo_data['Wi-Fi'].mean() * ureg.gigabyte),
        'Traffic to content provider': traffic_googfb_wifi * (allukvideo_data['Wi-Fi'].mean() * ureg.gigabyte),
    }
})
traffic_destination

,4G,Wi-Fi
Traffic to ISP edge cache,3372362776.4027762 gigabyte,38907473406.39639 gigabyte
Traffic to content provider,6407489275.165275 gigabyte,27464098875.10334 gigabyte


### Edge, Metro, Core

We now calculate the total energy consumption of the three sections of the internet.

In [32]:
internet_energy = pd.DataFrame({
    '4G': {
        'CPE & AN': cpe_an_4g,
        # All traffic goes to the edge
        'Edge': (allukvideo_data['4G'].mean() * ureg.gigabyte) * i_internet['edge'],
        'Metro': traffic_destination['4G']['Traffic to content provider'] * i_internet['metro'],
        'Core': traffic_destination['4G']['Traffic to content provider'] * i_internet['core'],
    },
    'Wi-Fi': {
        'CPE & AN': cpe_an_wifi,
        # All traffic goes to the edge
        'Edge': (allukvideo_data['Wi-Fi'].mean() * ureg.gigabyte) * i_internet['edge'],
        'Metro': traffic_destination['Wi-Fi']['Traffic to content provider'] * i_internet['metro'],
        'Core': traffic_destination['Wi-Fi']['Traffic to content provider'] * i_internet['core'],
    }
})

# Format nicely for display
internet_energy_display = pd.DataFrame({
    '4G': {
        'CPE & AN': f'{internet_energy["4G"]["CPE & AN"].to("gigawatt hours"):,.0f~H}',
        'Edge': f'{internet_energy["4G"]["Edge"].to("gigawatt hours"):,.0f~H}',
        'Metro': f'{internet_energy["4G"]["Metro"].to("gigawatt hours"):,.0f~H}',
        'Core': f'{internet_energy["4G"]["Core"].to("gigawatt hours"):,.0f~H}',
        'Total': f'{internet_energy["4G"].sum().to("gigawatt hours"):,.0f~H}',
    },
    'Wi-Fi': {
        'CPE & AN': f'{internet_energy["Wi-Fi"]["CPE & AN"].to("gigawatt hours"):,.0f~H}',
        'Edge': f'{internet_energy["Wi-Fi"]["Edge"].to("gigawatt hours"):,.0f~H}',
        'Metro': f'{internet_energy["Wi-Fi"]["Metro"].to("gigawatt hours"):,.0f~H}',
        'Core': f'{internet_energy["Wi-Fi"]["Core"].to("gigawatt hours"):,.0f~H}',
        'Total': f'{internet_energy["Wi-Fi"].sum().to("gigawatt hours"):,.0f~H}',
    }
})
internet_energy_display

,4G,Wi-Fi
CPE & AN,983 GW hr,"2,853 GW hr"
Edge,8 GW hr,57 GW hr
Metro,13 GW hr,55 GW hr
Core,33 GW hr,141 GW hr
Total,"1,037 GW hr","3,106 GW hr"


## Results

We now put this all together into a concluding results statement.

### Energy consumption

In [35]:
video_streaming_energy = pd.DataFrame({
    '4G': {
        'Total': internet_energy['4G'].sum(),
        'Intensity': internet_energy['4G'].sum() / (allukvideo_time['4G'].mean() * ureg.hours),
    },
    'Wi-Fi': {
        'Total': internet_energy['Wi-Fi'].sum(),
        'Intensity': internet_energy['Wi-Fi'].sum() / (allukvideo_time['Wi-Fi'].mean() * ureg.hours),
    },
    'Total': {
        'Total': internet_energy['4G'].sum() + internet_energy['Wi-Fi'].sum(),
        'Intensity': (internet_energy['4G'].sum() + internet_energy['Wi-Fi'].sum()) / allukvideo_time_total
    }
})

# As a percentage of UK generation
ukgeneration_2019 = 324.8 * ureg.terawatthours  # Source [20]
percentageGeneration = (video_streaming_energy['Total']['Total'].to(
    'gigawatthours') / ukgeneration_2019) / 1000

output = (
    f'We estimate {allukvideo_time_total.to("hours").to_compact().magnitude:,.1f}bn '
    f'hours of video were streamed in the UK in 2019 generating '
    f'{allukvideo_data_total.to("exabyte"):,.1f~H} of data and consuming '
    f'a total of {video_streaming_energy["Total"]["Total"].to("terawatthours"):,.1f~H} of electricity, or '
    f'{percentageGeneration.magnitude:.1%} of total electricity generation. '
    f'By connection type, energy consumption was split {video_streaming_energy["4G"]["Total"].to("terawatthours"):,.1f~H} '
    f'over 4G and {video_streaming_energy["Wi-Fi"]["Total"].to("terawatthours"):,.1f~H} over Wi-Fi. '
    f'Video streaming over 4G is twice as energy intensive per streaming hour '
    f'({video_streaming_energy["4G"]["Intensity"].to("kilowatthours/hour"):,.3f~H}) compared to Wi-Fi '
    f'({video_streaming_energy["Wi-Fi"]["Intensity"].to("kilowatthours/hour"):,.3f~H}).'
)
output

'We estimate 39.2bn hours of video were streamed in the UK in 2019 generating 76.2 EB of data and consuming a total of 4.1 TWh of electricity, or 1.3% of total electricity generation. By connection type, energy consumption was split 1.0 TWh over 4G and 3.1 TWh over Wi-Fi. Video streaming over 4G is twice as energy intensive per streaming hour (0.206 kWh/hr) compared to Wi-Fi (0.091 kWh/hr).'

### Carbon footprint

Having calculated the energy intensity, we can apply the 2019 electricity carbon conversion factor to calculate the carbon intensity of an hour of video streaming.

In [34]:
conversion_factor = 0.254 # Source [21]
carbon_per_streaming_hour = video_streaming_energy['Total']['Intensity'].magnitude * conversion_factor
print(f'Carbon intensity of use-stage networking for video streaming: {carbon_per_streaming_hour:,.3f} kgCO2/hour')

Carbon intensity of use-stage networking for video streaming: 0.027 kgCO2/hour


## References

1. UKOM (2019) Q4 2019 UK Digital Market Overview report. Available from: https://ukom.uk.net/uploads/files/news/ukom/174/UKOM_Digital_Marketing_Overview_December_2019_final.pdf

2. Ofcom (2020) Online Nation – 2020 report. Available from: https://www.ofcom.org.uk/__data/assets/pdf_file/0027/196407/online-nation-2020-report.pdf

3. Solsman, J.E. (2018) Normally secretive Netflix inches back the curtain on how subscribers stream. 7 March 2018. CNET. Available from: https://www.cnet.com/news/netflix-shares-streaming-data-by-device-country-mobile-wi-fi-movies-tv/

4. Hindy, J. (2019) How much data does YouTube actually use? 30 June 2019. Android Authority. Available from: https://www.androidauthority.com/how-much-data-does-youtube-use-964560/

5. Sandvine (2020) The Mobile Internet Phenomena Report. Available from: https://www.sandvine.com/download-report-mobile-internet-phenomena-report-2020-sandvine

6. Sandvine (2019) The Global Internet Phenomena Report. Available from: https://www.sandvine.com/global-internet-phenomena-report-2019

7. Ofcom (2019) Connected Nations 2019. 20 December 2019. Ofcom. Available from: https://www.ofcom.org.uk/research-and-data/multi-sector-research/infrastructure-research/connected-nations-2019/main-report

8. Chee, F.Y. (2020) YouTube, Amazon Prime forgo streaming quality to relieve European networks. Reuters. 20 March. Available from: https://uk.reuters.com/article/us-health-coronavirus-youtube-exclusive-idUKKBN2170OP

9. Ofcom (2019) Media Nations: UK 2019. Available from: https://www.ofcom.org.uk/__data/assets/pdf_file/0019/160714/media-nations-2019-uk-report.pdf

10. Office for National Statistics (2019) Overview of the UK population. 23 August 2019. Available from: https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/populationestimates/articles/overviewoftheukpopulation/august2019

11. Coroama, V.C., Schien, D., Preist, C. & Hilty, L.M. (2015) The Energy Intensity of the Internet: Home and Access Networks. In: Lorenz M. Hilty & Bernard Aebischer (eds.). ICT Innovations for Sustainability. Advances in Intelligent Systems and Computing. 2015 Cham, Springer International Publishing. pp. 137–155. Available from: doi:10.1007/978-3-319-09228-7_8.

12. Schien, D., Coroama, V.C., Hilty, L.M. & Preist, C. (2015) The Energy Intensity of the Internet: Edge and Core Networks. In: Lorenz M. Hilty & Bernard Aebischer (eds.). ICT Innovations for Sustainability. Advances in Intelligent Systems and Computing. 2015 Cham, Springer International Publishing. pp. 157–170. Available from: doi:10.1007/978-3-319-09228-7_9.

13. Masanet, E., Shehabi, A., Lei, N., Smith, S., et al. (2020) Recalibrating global data center energy-use estimates. Science. 367 (6481), 984–986. Available from: doi:10.1126/science.aba3758.

14. Nissen, N.F. (2007) EuP Preparatory Study Lot 6 “Standby and Off-mode Losses. Available from: https://www.eup-network.de/fileadmin/user_upload/Produktgruppen/Lots/Final_Documents/Los_06_final_report.pdf

15. Pihkola, H., Hongisto, M., Apilo, O. & Lasanen, M. (2018) Evaluating the Energy Consumption of Mobile Data Transfer—From Technology Development to Consumer Behaviour and Life Cycle Thinking. Sustainability. 10 (7), 2494. Available from: doi:10.3390/su10072494.

16. Apple (2019) iPhone 11 Pro - Technical Specifications. 2019. Apple. Available from: https://www.apple.com/iphone-11-pro/specs/

17. Espósito, F. (2019) iPhone 11 battery size confirmed in new regulatory filings. 9to5Mac. Available from: https://9to5mac.com/2019/09/17/iphone-11-and-iphone-11-pro-battery-size/

18. Apple (n.d.) iPhone - Battery Test Information - Apple. Available from: https://www.apple.com/iphone/battery.html

19. Aslan, J., Mayers, K., Koomey, J.G. & France, C. (2018) Electricity Intensity of Internet Data Transmission: Untangling the Estimates: Electricity Intensity of Data Transmission. Journal of Industrial Ecology. 22 (4), 785–798. Available from: doi:10.1111/jiec.12630.

20. Department for Business, Energy & Industrial Strategy (2020) UK energy in brief 2020. 30 July 2020. GOV.UK. Available from: https://www.gov.uk/government/statistics/uk-energy-in-brief-2020

21. Department for Business, Energy & Industrial Strategy (2020) Greenhouse gas reporting: conversion factors 2019. 28 July 2020. GOV.UK. Available from: https://www.gov.uk/government/publications/greenhouse-gas-reporting-conversion-factors-2019

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b38c2c00-d173-47f7-8844-adf84ba73830' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>